#  Behaviors

During the last practical sessions, we have seen how to send commands to the robot's actuators (wheels) and how to read sensor values (proximeters). This week we'll see how to connect perception to action by programming a few behaviors.

## Preliminary steps

**Important** : First make a copy of this notebook to work on it (it is useful in case I have to modify this one). To do so, follow the procedure explained at the start of the last practical session notebook.

The interface allowing to control the E-Puck in Python has changed a bit since the last practical session (nothing huge, don't worry). Let's first connect this notebook to the Epuck robot in V-REP. To do so, first start V-REP and load the `epuck.ttt` scene as we did during the last session. Then, connect to the simulator and to the robot by executing (remember that you have to press `Shift+Enter` in the code cells below to execute them) :

In [1]:
# necessary stuff to set the paths:
import sys
sys.path.append("..")

# Import and create the connection to the simulator:
from vrep.simulator import Simulator
simulator = Simulator()

# connect to the E-puck robot of the running V-REP scene
epuck = simulator.get_epuck()

# Start the simulator (equivalent to pressing the "play" button in vrep). To stop it, replace "start" by "stop"
simulator.start()

Now your are in the same situation as in the previous session. Each time you will have to restart this notebook (e.g. because you lost the connection with V-REP), you'll have to re-execute the cell above. 

To check that the connection is effectively working, start the simulator and see if you can read the proximeter values:

In [ ]:
epuck.proximeters()

If you see values, everything is working (otherwise call me). Remember that they correspond to each of the 8 proximeter values. The Epuck's proximeters are indexed from 0 to 7 according to the following scheme (indexes in red, the black arrows indicating the front of the robot). 

![Epuck proximeter groups](files/epuck_prox_groups.png)

For example, the two front proximeters are at index 2 and 3, whereas the two rear one are at 6 and 7. 

Each proximeter is able to detect obstacles up to 2 meters away from the robot. If it detects nothing, the value is the maximum distance (2000 millimeters). Note that this is different from the last session settings, where the proximeters were returning 0 when no obstacle was detected. Now it returns 2000 instead (we can configure this value, ask me if you want to know how).

Another change is how to send commands to the robot wheels. Now we do it like this (the command we have seen during the last session are still working, but it is recommended to use these new ones):

In [3]:
# Setting the left wheel speed to 0.2 rad/s:
epuck.left_spd = 0.2

# the robot should turn slowly around itself

In [4]:
# Setting the right wheel speed to 0.2 rad/s:
epuck.right_spd = 0.2

# the robot should go forward (since both wheels have the same speed)

In [16]:
# Stopping the robot:
epuck.left_spd = 0.
epuck.right_spd = 0.

Using that kind of syntax allows reading the speed of a wheel using a very similar code:

In [ ]:
# read the current speed of the left wheel:
epuck.right_spd

It is also possible to increment the value of a wheel speed:

In [15]:
# Add 0.1 to the current speed of the left wheel:
epuck.left_spd +=  0.1

Try executing the last cell several times to increase the left wheel speed again and again. To decrement instead of increment, you just have to replace `+=` by `-=`.

## Sensorimotor loops

We will program behaviors as sensorimotor loops, i.e. the repetition at a high frequency of the following steps:
- read sensor values
- process information
- send actuator commands

For example, let's consider that we want to program a STOP behavior, such that the robot stops when it arrives at 10cm of an obstacle. In that case, the three steps above would correspond to:
- *read sensor values:* read the values of the two front proximeters of the robot,
- *process information:* check whether one of this value is lesser than 100 (remember that proximeter values are in millimeters),
- *send actuator commands:* if a proximeter indicates a value lesser than 100, set both wheel velocities to 0, otherwise set them to a positive value.



Let's see how we can program each of those three steps. First, *read the sensor values*:

In [18]:
proxs = epuck.proximeters()

The line above reads the values of the proximeters and store them in a variable called `proxs`. Now `proxs` contains the value of the proximeters at the time you have executed the line above (meaning that the values in `proxs` are not updated automatically when the robot moves). Let's print what has been stored:

In [ ]:
proxs

Next we have to *process this information* to check if the front proximeters are detecting an obstacle at less than 100mm. To do so, we need to specifically access the front proximeters of the robot, which are indexed by 2 and 3 according to the scheme we have seens above. We can therefore access their values using:

In [ ]:
# Value of the proximeter number 2:
proxs[2]

In [ ]:
# Value of the proximeter number 3:
proxs[3]

Now we have to check if one of these two values is lesser than 100. In logical terms, they are if `proxs[2]` is lesser than 100 **OR** `proxs[3]` is lesser than 100. In Python we write it like this:

In [6]:
proxs[2] < 100 or proxs[3] < 100

False

This last expression returns `True` if at least one of the conditions surrounding `or` are true. Otherwise it returns `False` (i.e. when both conditions are false).

Finally, the last step is to *send the actuator command* according to what has been perceived from the environment. Here we want to stop the robot if `proxs[2]` or `proxs[3]` is lesser than 100 and to move forward otherwise. This can be done using an *if-then-else* structure which is very common in programming languages: 

In [7]:
if proxs[2] < 100 or proxs[3] < 100:  # The two indented lines below are executed only if the condition is True
    # Set both wheel speeds to 0 (stop the robot)
    epuck.left_spd = 0.
    epuck.right_spd = 0.
else:  # if the condition is False, the two indented lines below are executed instead
    # Set both wheel speeds to 10 (move forward)
    epuck.left_spd = 10.
    epuck.right_spd = 10. 

Those three steps have to be repeated in a loop so that the robot continuously read its sensors and acts accordingly (either stop or move forward). To do so we use another common programming structure called a `while` loop:

In [ ]:
while True:  # basically means "repeat forever the indented lines below (which are the exact same ones as above)"
    proxs = epuck.proximeters()
    if proxs[2] < 100 or proxs[3] < 100:
        epuck.left_spd = 0.
        epuck.right_spd = 0.
    else:
        epuck.left_spd = 10.
        epuck.right_spd = 10.        

If the front of your robot was more that 10cm away from an obstacle, you should observe it moving forward and stops when close to an obstacle (otherwise stop and start the simulation again). 

Please note that the above loop run forever: the star on the left of the last cell indicates it is actually still running, and therefore you can't execute other cells in this notebook. To stop it, you have to press the "stop"-like button at the top of this notebook. You will see cryptic messages printed below the cell: basically they show you where the code has actually been stopped.

## New functionalities

Before starting the exercices where you will program a few behaviors, this section present a set of new functionalities which are practical to send command to the robot's actuators and to read values from sensors. They will be useful to do the exercices, so be sure to understand all of them (ask if needed).

### Actuators

Instead of setting the left and right wheel speed, you have the possibility to directly control the robot's forward speed and the rotation speed using:

In [20]:
# Move forward at 0.2 meter per second
epuck.fwd_spd = 0.2

In [21]:
# Stop moving forward
epuck.fwd_spd = 0.

In [22]:
# Rotate the robot at 1 radian per second
epuck.rot_spd = 1.

In [23]:
# Rotate the robot at -1 radian per second
epuck.rot_spd = -1.

In [25]:
# Move backward at 0.1 m/s (it will keep the last rotation speed)
epuck.fwd_spd = -0.1

In [26]:
# Stop moving forward (it will keep the last rotation speed)
epuck.fwd_spd = 0.

In [27]:
# Stop rotate
epuck.rot_spd = 0.

Whatever the command you use (`left_spd`, `right_spd`, `fwd_spd` or `rot_spd`), the other value reading will stay coherent. For example, let's make the robot makes a small circle using `left_spd` and `right_spd`:

In [28]:
epuck.left_spd = 1.
epuck.right_spd = 0.6

We can then check what are the corresponding forward and rotation speed (they are automatically computed internally):

In [ ]:
epuck.fwd_spd, epuck.rot_spd

### Sensors

#### Proximeter properties

Accessing particular proximeter values through their indexes, as we have done above, can sometime be fastidious. This is why a  number of aliases can be used to access groups of proximeters. You can see them with:

In [ ]:
epuck._prox_aliases

This will display some text associating some aliases with some groups of parameter indexes. In the lines above, the character string on the left indicates the alias name, and the list of integers on the right the corresponding proximeter indexes. For example, the alias `'front-right'` correspond to the proximeters 3, 4 and 5 (which are indeed at the right side of the robot).
Those aliases can be used to easily access particular proximeter values, for example:

In [ ]:
epuck.proximeters('front')

returns only the values of the front proximeters (indexes 2 and 3). Note that without any alias indicated, `epuck.proximeters()` is equivalent to `epuck.proximeters('all')` (i.e. returns the values of all the proximeters).

You can also access to more pre-processed information about the proximeters. For example:

In [ ]:
epuck.min_distance()

returns the minimum distance perceived from all the proximeters. You can also specify a group alias, for example:

In [ ]:
epuck.min_distance('front')

returns the minimum distance perceived from the two front proximeters (2 and 3).

Another useful function is:

In [ ]:
epuck.min_index()

which returns the index of the proximeter indicating the shortest distance. This can for example be useful to find in which direction is the closest obstacle.

Here again you can specify a group alias, e.g:

In [ ]:
epuck.min_index('all-but-rear')

does the same thing without considering the two rear proximeters. 

#### Floor sensor

The E-puck is also equipped with floor sensors, allowing the perception of a black line drawn on the floor. On the robot representation image above, the three floor sensors are the three grey squares on the front of the robot (i.e. at the bottom of the robot image, the front being indicated by the black arrow). Each of these sensors can have two states: detecting a line, or not.

In the V-REP scene you are working on, a line is drawn on the floor (zoom out if you don't see it). Place your robot on it so that it detects a line with at least one of its floor sensors (you can see the robot representation by clicking on the E-Puck when the simulation is running, when a floor sensor is black it is detecting a line). Then execute:

In [ ]:
epuck.floor_sensor()

This returns three boolean values (i.e. each one is either `True` or `False`) indicating if a line was detected by the left, middle and right floor sensor, in that order (`True` means a line is detected, `False` means nothing is detected).

You can access specific floor sensor values the same way as for the proximeters: floor sensors are indexed by 0 (left sensor), 1 (middle sensor) and 2 (right sensor). Therefore, the value of the middle sensor, for example, is obtained with: 

In [ ]:
epuck.floor_sensor()[1]

Another way to proceed is the following:

In [ ]:
left, middle, right = epuck.floor_sensor()

That way, `left` (resp. `middle` and `right`) contains the value (`True` or `False`) of the left floor sensor (resp. middle and right sensors). Try it:

In [ ]:
left

If you want to convert the boolean into an integer, you can write:

In [ ]:
int(left)

This will return 0 if `left` is `False` (no line detected) and 1 if `left` is `True` (line detected).  This can be useful to convert the values returned by the floor sensors into wheel speeds.

### Generating random numbers

Sometimes it can be useful to generate random movements of the robot, e.g. to program an exploration behavior. To generate a random number, you have to add the following line at the beginning of your code:
    
    from numpy.random import rand
    
Then, calling `rand()` will generate a random number between 0 and 1. Using multiplication and addition, you can generate a random number in any range. For example:

In [ ]:
from numpy.random import rand

# generate a random number between 0 and 1:
rand()

In [ ]:
# generate a random number between -2 and 2:
(rand() * 4.) - 2

In [ ]:
# generate a random number between a and b (you can try changing the value of a and b below):
a = 1
b = 5
(rand() * (b - a)) + a

## Exercices

**Q1:** Modify the STOP behavior implemented above such that the speed of the robot is proportional to the distance from the closest obstacle in front of it (the shorter the distance, the lower the speed). The robot has to stop before hitting the obstacle.

**Q2:** Program a behavior such that the robot makes a random movement when an obstacle is closer than 10cm. Use the random number generator introduced above.

**Q3:** Program a line following behavior using the floor sensors. When initially positioned on a line, the robot should follow it.

**Q4:** Program an obstacle avoidance behavior by using `epuck.min_index` and setting the rotation speed accordingly. Then, to test its robustness, add obstacles in the V-REP scene, e.g. pillars (drag and drop them from `Models/infrastructure/walls/20cm high walls`).

**Q5: ** Program an object pushing behavior, using a similar technique as in the previous question. To test it, add pushable objects in the scene, e.g. cups (drag and drop them from `Models/households`).

**Q6: ** Be creative and program a behavior of your choice.